In [1]:
import geopandas
import pytz
from datetime import datetime
import os
import shutil
import wget

In [2]:
def get_frp_fn_url(dt):
    tt = dt.timetuple()
    month = str(tt.tm_mon).zfill(2)
    day = str(tt.tm_mday).zfill(2)
    yr = str(tt.tm_year)
    fn = 'hms_fire{}{}{}.zip'.format(yr, month, day)
    url = 'https://satepsanone.nesdis.noaa.gov/pub/FIRE/web/HMS/Fire_Points/Shapefile/{}/{}/{}'.format(yr, month, fn)
    return fn, url

In [3]:
def get_frp(dt, frp_dir):
    fn, url = get_frp_fn_url(dt)
    frp_shape_fn = frp_dir + fn
    print(frp_shape_fn)
    if os.path.exists(frp_shape_fn):
        print("{} already exists".format(fn))
    else:
        print('DOWNLOADING frp: {}'.format(fn))
        filename = wget.download(url, out=frp_dir)
        shutil.unpack_archive(filename, frp_dir)
    frp = geopandas.read_file(frp_shape_fn)
    return frp


In [4]:
dt_str = '2023/04/04 14:50'
dt = pytz.utc.localize(datetime.strptime(dt_str, '%Y/%m/%d %H:%M'))
print(dt)
get_frp(dt, "/home/rey/projects/GOES_FRP/frp_shapefiles/")


2023-04-04 14:50:00+00:00
/home/rey/projects/GOES_FRP/frp_shapefiles/hms_fire20230404.zip
hms_fire20230404.zip already exists


,Lon,Lat,YearDay,Time,Satellite,Method,Ecosystem,FRP,geometry
0,-66.744949,18.077087,2023094,0506,NOAA 20,VIIRS,93,2.219,POINT (-66.74495 18.07709)
1,-68.066635,46.766396,2023094,0552,SUOMI NPP,VIIRS,30,6.170,POINT (-68.06664 46.7664)
2,-77.398834,26.739498,2023094,0558,SUOMI NPP,VIIRS,93,1.732,POINT (-77.39883 26.7395)
3,-77.392616,26.739120,2023094,0558,SUOMI NPP,VIIRS,93,1.379,POINT (-77.39262 26.73912)
4,-79.201447,21.899944,2023094,0558,SUOMI NPP,VIIRS,41,1.362,POINT (-79.20145 21.89994)
...,...,...,...,...,...,...,...,...,...
47270,-90.152489,19.113523,2023095,0556,GOES-EAST,FDC,34,29.181,POINT (-90.15249 19.11352)
47271,-90.048340,19.091976,2023095,0556,GOES-EAST,FDC,34,73.806,POINT (-90.04834 19.09198)
47272,-84.134743,15.025342,2023095,0556,GOES-EAST,FDC,22,22.093,POINT (-84.13474 15.02534)
47273,-80.857155,22.808207,2023095,0556,GOES-EAST,FDC,93,18.415,POINT (-80.85716 22.80821)
